In [2]:
# TRAINING

import numpy as np
import os
import cv2
import time
import yaml
import torch
from ultralytics import YOLO

# define some parameters
CONFIDENCE = 0.5
font_scale = 1
thickness = 1

# loading the YOLOv8 model with the default weight file
model = YOLO("yolov8n.pt")

# training the model
results = model.train(data = "coco128.yaml", epochs = 1)

New https://pypi.org/project/ultralytics/8.0.123 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.120  Python-3.11.4 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=tor

               frisbee        128          5      0.678        0.8       0.76      0.688
                  skis        128          1      0.548          1      0.995      0.497
             snowboard        128          7      0.757      0.714      0.744      0.494
           sports ball        128          6      0.701      0.403      0.493      0.285
                  kite        128         10      0.807      0.422      0.568      0.184
          baseball bat        128          4      0.628      0.442      0.419        0.2
        baseball glove        128          7      0.648      0.429      0.429      0.273
            skateboard        128          5       0.78        0.6        0.6      0.427
         tennis racket        128          7      0.725      0.387      0.487      0.359
                bottle        128         18      0.532       0.38      0.366      0.211
            wine glass        128         16      0.676      0.392      0.573      0.333
                   cu

In [3]:
# Analysing model's parameters
model_parameters = torch.load("C:\\Users\\evasu\\All_I_can_eat\\runs\\detect\\train4\\weights\\best.pt")

# Printing the parameters
print(model_parameters)

{'epoch': -1, 'best_fitness': None, 'model': DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
 

In [4]:
# PREDICTIONS

# loading all the class labels (objects)
with open("C:\\Users\\evasu\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\ultralytics\\datasets\\coco128.yaml", "r", encoding = "utf8") as file:
    labels = list(yaml.safe_load(file)['names'].values())

# generating colors for each object for later plotting
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

# loading the image for prediction
path_name = "C:\\Users\\evasu\\Downloads\\fante.jpg"
image = cv2.imread(path_name)
file_name = os.path.basename(path_name) # "dog.jpg"
filename, ext = file_name.split(".") # "dog", "jpg"

# measure how much it took in seconds
start = time.perf_counter()
# run inference on the image 
# see: https://docs.ultralytics.com/modes/predict/#arguments for full list of arguments
results = model.predict(image, conf=CONFIDENCE)[0]
time_took = time.perf_counter() - start
print(f"Time took: {time_took:.2f}s")
print(results.boxes.data)


0: 480x640 1 elephant, 165.1ms
Speed: 10.8ms preprocess, 165.1ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)


Time took: 0.27s
tensor([[1.3020e+02, 1.1309e+02, 1.0691e+03, 8.6381e+02, 8.7161e-01, 2.0000e+01]])


In [ ]:
# loop over the detections
for data in results.boxes.data.tolist():
    # get the bounding box coordinates, confidence, and class id 
    xmin, ymin, xmax, ymax, confidence, class_id = data
    # converting the coordinates and the class id to integers
    xmin = int(xmin)
    ymin = int(ymin)
    xmax = int(xmax)
    ymax = int(ymax)
    class_id = int(class_id)

    # draw a bounding box rectangle and label on the image
    color = [int(c) for c in colors[class_id]]
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=1, thickness=thickness)
    text = f"{labels[class_id]}: {confidence:.2f}"
    # calculate text width & height to draw the transparent boxes as background of the text
    (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
    text_offset_x = xmin
    text_offset_y = ymin - 5
    box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
    overlay = image.copy()
    cv2.rectangle(overlay, box_coords[0], box_coords[1], color=1, thickness=cv2.FILLED)
    # add opacity (transparency to the box)
    image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
    # now put the text (label: confidence %)
    cv2.putText(image, text, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=font_scale, color=(0, 0, 0), thickness=thickness)

# display output image
cv2.imshow("Image", image)
cv2.waitKey(0)
# save output image to disk
cv2.imwrite(filename + "_yolo8." + ext, image)